### SQLChatMessageHistory
* 채팅메세지를 SQL 데이터베이스 저장하는 클래스

**주요특징**
* SQL 데이터베이스를 사용해 채팅 기록을 영구적으로 저장
* 세션 ID를 통한 대화 구분 및 관리 기능
* SQLAlchemy를 사용해 다양한 데이터베이스 사용 가능
* 메세지 CRUD 가능

* session_id : 사용자 이름, 이메일 같이 고유 식별자
* connection: 데이터베이스 연결을 지정하는 문자


In [2]:
from langchain_community.chat_message_histories import SQLChatMessageHistory

chat_message_history = SQLChatMessageHistory(
    session_id ="sql_history",
    connection="sqlite:///sqlite.db"
)

In [3]:
chat_message_history.add_user_message(
    "안녕? 나는 bear"
)

In [4]:
chat_message_history.add_ai_message(
    "안녕 bear, 만나서 반가워"
)

In [5]:
chat_message_history.messages

[HumanMessage(content='안녕? 나는 bear', additional_kwargs={}, response_metadata={}),
 AIMessage(content='안녕 bear, 만나서 반가워', additional_kwargs={}, response_metadata={})]

In [8]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI Assistant"),
        
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}")
    ]
)

chain = prompt | ChatOpenAI(temperature=0, model="gpt-4o") | StrOutputParser()

In [9]:
def get_chat_history(user_id,conversation_id):
    return SQLChatMessageHistory(
        table_name=user_id,
        session_id=conversation_id,
        connection="sqlite:///sqlite.db"
    )

In [ ]:
from langchain_core.runnables.history import RunnableWithMessageHistory

chain_with_history = RunnableWithMessageHistory(
    chain,
)